In [1]:
import os
import glob
import shutil

import pandas as pd
import numpy as np
import pylab as pl
import seaborn as sns

import re
import scipy.stats as spstats
import json
from pipeline.python.classifications import experiment_classes as util

/home/julianarhee/Repositories/2p-pipeline/pipeline/python/classifications/experiment_classes.py:18: UserWarning: 
This call to matplotlib.use() has no effect because the backend has already
been chosen; matplotlib.use() must be called *before* pylab, matplotlib.pyplot,
or matplotlib.backends is imported for the first time.

The backend was *originally* set to 'module://ipykernel.pylab.backend_inline' by the following code:
  File "/home/julianarhee/anaconda2/envs/pipeline/lib/python2.7/runpy.py", line 174, in _run_module_as_main
    "__main__", fname, loader, pkg_name)
  File "/home/julianarhee/anaconda2/envs/pipeline/lib/python2.7/runpy.py", line 72, in _run_code
    exec code in run_globals
  File "/home/julianarhee/anaconda2/envs/pipeline/lib/python2.7/site-packages/ipykernel_launcher.py", line 16, in <module>
    app.launch_new_instance()
  File "/home/julianarhee/anaconda2/envs/pipeline/lib/python2.7/site-packages/traitlets/config/application.py", line 658, in launch_instance
   

/home/julianarhee/Repositories/2p-pipeline/pipeline/python/traces/get_traces.py:96: UserWarning: 
This call to matplotlib.use() has no effect because the backend has already
been chosen; matplotlib.use() must be called *before* pylab, matplotlib.pyplot,
or matplotlib.backends is imported for the first time.

The backend was *originally* set to 'module://ipykernel.pylab.backend_inline' by the following code:
  File "/home/julianarhee/anaconda2/envs/pipeline/lib/python2.7/runpy.py", line 174, in _run_module_as_main
    "__main__", fname, loader, pkg_name)
  File "/home/julianarhee/anaconda2/envs/pipeline/lib/python2.7/runpy.py", line 72, in _run_code
    exec code in run_globals
  File "/home/julianarhee/anaconda2/envs/pipeline/lib/python2.7/site-packages/ipykernel_launcher.py", line 16, in <module>
    app.launch_new_instance()
  File "/home/julianarhee/anaconda2/envs/pipeline/lib/python2.7/site-packages/traitlets/config/application.py", line 658, in launch_instance
    app.start()
  Fi

In [2]:
from pipeline.python.utils import label_figure, natural_keys

In [3]:
%matplotlib notebook

In [19]:
def load_traces(animalid, session, fovnum, curr_exp, traceid='traces001',
               responsive_test='ROC', responsive_thr=0.05, response_type='dff'):
    
    # Load experiment neural data
    fov = 'FOV%i_zoom2p0x' % fovnum
    if curr_exp=='blobs':
        exp = util.Objects(animalid, session, fov, traceid=traceid)
    else:
        exp = util.Gratings(animalid, session, fov, traceid=traceid)
    
    exp.load(trace_type=response_type, update_self=True, make_equal=False)
    
    labels = exp.data.labels.copy()

    # Get stimulus config info
    sdf = exp.data.sdf
    if curr_exp == 'blobs':
        sdf = reformat_morph_values(sdf)
    n_conditions = len(sdf['size'].unique())

    # Get responsive cells
    responsive_cells, ncells_total = exp.get_responsive_cells(response_type=response_type,\
                                                              responsive_test=responsive_test,
                                                              responsive_thr=responsive_thr)
    
    traces = exp.data.traces[responsive_cells]

    return traces, labels, sdf

def reformat_morph_values(sdf):
    control_ixs = sdf[sdf['morphlevel']==-1].index.tolist()
    sizevals = np.array([round(s, 1) for s in sdf['size'].unique() if s not in ['None', None] and not np.isnan(s)] )
    sdf.loc[sdf.morphlevel==-1, 'size'] = pd.Series(sizevals, index=control_ixs)
    sdf['size'] = [round(s, 1) for s in sdf['size'].values]

    return sdf


In [20]:
def traces_to_trials(traces, labels, epoch='stimulus'):
    '''
    Returns dataframe w/ columns = roi ids, rows = mean response to stim ON per trial
    Last column is config on given trial.
    '''
    s_on = int(labels['stim_on_frame'].mean())
    n_on = int(labels['nframes_on'].mean())

    roi_list = traces.columns.tolist()
    trial_list = np.array([int(trial[5:]) for trial, g in labels.groupby(['trial'])])
    if epoch=='stimulus':
        mean_responses = pd.DataFrame(np.vstack([np.nanmean(traces.iloc[g.index[s_on:s_on+n_on]], axis=0)\
                                            for trial, g in labels.groupby(['trial'])]),
                                             columns=roi_list, index=trial_list)
    elif epoch == 'baseline':
        mean_responses = pd.DataFrame(np.vstack([np.nanmean(traces.iloc[g.index[0:s_on]], axis=0)\
                                            for trial, g in labels.groupby(['trial'])]),
                                             columns=roi_list, index=trial_list)

    condition_on_trial = np.array([g['config'].unique()[0] for trial, g in labels.groupby(['trial'])])
    mean_responses['config'] = condition_on_trial

    return mean_responses

# Get aggregate datasets

In [11]:
from pipeline.python.classifications import get_dataset_stats as gd
import cPickle as pkl

options = ['-t', 'traces001']
optsE = gd.extract_options(options)

rootdir = optsE.rootdir
aggregate_dir = optsE.aggregate_dir
fov_type = optsE.fov_type
traceid = optsE.traceid
response_type = 'dff'
print aggregate_dir

sdata_fpath = os.path.join(aggregate_dir, 'dataset_info.pkl')
if os.path.exists(sdata_fpath):
    with open(sdata_fpath, 'rb') as f:
        sdata = pkl.load(f)
else:
    sdata = gd.aggregate_session_info(traceid=optsE.traceid, trace_type=optsE.trace_type, 
                                       state=optsE.state, fov_type=optsE.fov_type, 
                                       visual_areas=optsE.visual_areas,
                                       blacklist=optsE.blacklist, 
                                       rootdir=optsE.rootdir)
    with open(sdata_fpath, 'wb') as f:
        pkl.dump(sdata, f, protocol=pkl.HIGHEST_PROTOCOL)

/n/coxfs01/julianarhee/aggregate-visual-areas


# Set data stat type

In [22]:
# Select response filters
responsive_test='ROC'
responsive_thr = 0.05
response_type = 'df'
always_exclude = ['20190426_JC078']


In [23]:
#### Load mean trial info for responsive cells
data_dir = os.path.join(aggregate_dir, 'data-stats')
#os.listdir(data_dir)

#### BLOB DATA
experiment = 'blobs'

load_data = False
data_desc = '%s_%s-%s_%s-thr-%.2f' % (experiment, traceid, response_type, responsive_test, responsive_thr)
data_outfile = os.path.join(data_dir, '%s.pkl' % data_desc)

if not os.path.exists(data_outfile):
    load_data = True
print(load_data)

print(data_desc)

True
blobs_traces001-df_ROC-thr-0.05


In [28]:
tmp_exclude = ['20190522_JC084']

In [30]:
if load_data:
    
    print("Getting data: %s" % experiment)
    print("Saving data to %s" % data_outfile)
    
    dsets = sdata[sdata['experiment']==experiment].copy()

    DATA = {}
    for (animalid, session, fovnum), g in dsets.groupby(['animalid', 'session', 'fovnum']):
        datakey = '%s_%s_fov%i' % (session, animalid, fovnum)
        if '%s_%s' % (session, animalid) in always_exclude:
            continue
            
        if '%s_%s' % (session, animalid) in tmp_exclude:
            continue

        # Load traces
        traces, labels, sdf = load_traces(animalid, session, fovnum, experiment, 
                                          traceid=traceid, response_type=response_type,
                                          responsive_test=responsive_test, responsive_thr=responsive_thr)
        # Calculate mean trial metric
        mean_responses = traces_to_trials(traces, labels)

        DATA['%s' % datakey] = {'data': mean_responses,
                                'sdf': sdf}
        
    # Save
    with open(data_outfile, 'wb') as f:
        pkl.dump(DATA, f, protocol=pkl.HIGHEST_PROTOCOL)
    print("Done!")
    

Getting data: blobs
Saving data to /n/coxfs01/julianarhee/aggregate-visual-areas/data-stats/blobs_traces001-df_ROC-thr-0.05.pkl
... [JC076|20190420|FOV1_zoom2p0x] creating blobs object
... getting data paths - name: blobs
['/n/coxfs01/2p-data/JC076/20190420/FOV1_zoom2p0x/combined_blobs_static']
['/n/coxfs01/2p-data/JC076/20190420/FOV1_zoom2p0x/combined_blobs_static']
... run: combined_blobs_static
... loading data array
/n/coxfs01/2p-data/JC076/20190420/FOV1_zoom2p0x/combined_blobs_static/traces/traces001_5ffca7_traces001_09c921_traces001_512e96_traces001_394d78_traces001_64e684_traces001_6bacb1/data_arrays/np_subtracted.npz
          color morphlevel   object  size stimtype  xpos  ypos yrot
config001  0.08         -1  control  None    image  None  None    0
config002   0.2         -1  control  None    image  None  None    0
config003  0.32         -1  control  None    image  None  None    0
config004  0.44         -1  control  None    image  None  None    0
config005  0.56         -1 

... getting data paths - name: blobs
['/n/coxfs01/2p-data/JC078/20190509/FOV1_zoom2p0x/combined_blobs_static']
['/n/coxfs01/2p-data/JC078/20190509/FOV1_zoom2p0x/combined_blobs_static']
... run: combined_blobs_static
... loading data array
/n/coxfs01/2p-data/JC078/20190509/FOV1_zoom2p0x/combined_blobs_static/traces/traces001_d242e7_traces001_37b7c0_traces001_dcf502_traces001_50495c_traces001_b5b103_traces001_31048d/data_arrays/np_subtracted.npz
          color morphlevel   object  size stimtype  xpos  ypos yrot
config001  0.08         -1  control  None    image  None  None    0
config002   0.2         -1  control  None    image  None  None    0
config003  0.32         -1  control  None    image  None  None    0
config004  0.44         -1  control  None    image  None  None    0
config005  0.56         -1  control  None    image  None  None    0
NP_subtracted offset was: 103.98
adding NP offset... (NP baseline offset: 374.32)
((268500, 295), (295,), ())
... updating self
... getting resp

          color morphlevel   object  size stimtype  xpos  ypos yrot
config001  0.08         -1  control  None    image  None  None    0
config002   0.2         -1  control  None    image  None  None    0
config003  0.32         -1  control  None    image  None  None    0
config004  0.44         -1  control  None    image  None  None    0
config005  0.56         -1  control  None    image  None  None    0
NP_subtracted offset was: 60.57
adding NP offset... (NP baseline offset: 426.31)
((313250, 192), (192,), ())
... updating self
... getting responsive cells (test: ROC, thr: 0.05')
-- stats: combined_blobs_static
['/n/coxfs01/2p-data/JC083/20190510/FOV1_zoom2p0x/combined_blobs_static/traces/traces001_54393b_traces001_0152b8_traces001_d41efc_traces001_cd9df6_traces001_aab980_traces001_84e961_traces001_ba9020/summary_stats/ROC/roc_results.pkl']
... [JC083|20190511|FOV1_zoom2p0x] creating blobs object
... getting data paths - name: blobs
['/n/coxfs01/2p-data/JC083/20190511/FOV1_zoom2p0x/co

NP_subtracted offset was: 12.62
adding NP offset... (NP baseline offset: 524.30)
((220725, 271), (271,), ())
... updating self
... getting responsive cells (test: ROC, thr: 0.05')
-- stats: combined_blobs_static
['/n/coxfs01/2p-data/JC090/20190605/FOV1_zoom2p0x/combined_blobs_static/traces/traces001_5afb30_traces001_ea6bc7_traces001_9747d8_traces001_70031f_traces001_ba4282_traces001_c2eec5_traces001_5c088e/summary_stats/ROC/roc_results.pkl']
... [JC091|20190602|FOV1_zoom2p0x] creating blobs object
... getting data paths - name: blobs
['/n/coxfs01/2p-data/JC091/20190602/FOV1_zoom2p0x/combined_blobs_static']
['/n/coxfs01/2p-data/JC091/20190602/FOV1_zoom2p0x/combined_blobs_static']
... run: combined_blobs_static
... loading data array
/n/coxfs01/2p-data/JC091/20190602/FOV1_zoom2p0x/combined_blobs_static/traces/traces001_573669_traces001_3d2687_traces001_17838a_traces001_452178_traces001_c84a17_traces001_faae98/data_arrays/np_subtracted.npz
          color morphlevel   object  size stimtyp

NP_subtracted offset was: -18.51
adding NP offset... (NP baseline offset: 758.05)
((425250, 560), (560,), ())


Traceback (most recent call last):
  File "/home/julianarhee/Repositories/2p-pipeline/pipeline/python/classifications/experiment_classes.py", line 1081, in load
    tmp_df = []
  File "/home/julianarhee/anaconda2/envs/pipeline/lib/python2.7/site-packages/pandas/core/ops.py", line 1241, in f
    return self._combine_const(other, na_op)
  File "/home/julianarhee/anaconda2/envs/pipeline/lib/python2.7/site-packages/pandas/core/frame.py", line 3543, in _combine_const
    raise_on_error=raise_on_error)
  File "/home/julianarhee/anaconda2/envs/pipeline/lib/python2.7/site-packages/pandas/core/internals.py", line 3197, in eval
    return self.apply('eval', **kwargs)
  File "/home/julianarhee/anaconda2/envs/pipeline/lib/python2.7/site-packages/pandas/core/internals.py", line 3091, in apply
    applied = getattr(b, f)(**kwargs)
  File "/home/julianarhee/anaconda2/envs/pipeline/lib/python2.7/site-packages/pandas/core/internals.py", line 1191, in eval
    result = handle_error()
  File "/home/julia

ERROR LOADING DATA


AttributeError: 'NoneType' object has no attribute 'labels'